# Tableau de bord

In [ ]:
# Paquets
from collections import namedtuple
from pathlib import Path
import dask
from dask.diagnostics import ProgressBar
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.xarray
from io import StringIO
import numpy as np
import pandas as pd
import panel as pn
from panel.viewable import Viewer
import param
from shapely.geometry import Point, LineString
import xarray as xr
import xclim as xc
from xclim import analog as xa
from datetime import datetime
import pickle
import joblib

import json

from core import utils, widgets, search
from core.constants import fut_col, hist_col, ana_col, quality_terms, quality_colors

config_path = Path('config.json')
if config_path.is_file():
    with open(config_path,encoding='utf-8') as config_file:
        config = json.load(config_file)

if not utils.check_version(config):
    print("Old config version detected. Removing cached files.")
    !rm density.obj
    !rm benchmark.obj
    !rm -rf ./cache/*
        
gv.extension('bokeh')
pn.extension(
    raw_css=[config["css"].replace('\\n','').replace('\n','')],
    loading_spinner='dots',
    loading_color='#00aa41'
)

# constants
# Dask. To make this dashboard slightly faster, change the "scheduler" argument to scheduler='processes' and num_workers=4 (for example)
# However the final webapp most likely won't have access to this kind of parallelism
dask.config.set(scheduler=config["options"]["dask_schedule"], temporary_directory='/notebook_dir/writable-workspace/tmp')
try:
    curr_dir = Path(__file__).parent
except NameError:  # When running as a notebook "__file__" isn't defined.
    curr_dir = Path('.')
cities_file = f'{curr_dir}/cities_tmp.geojson'

# Projection
biasadjust = config["options"]["biasadjust"] # scaling or dqm the method used for the annual adjustment method

# Random city on load
init_rand_city = config["options"]["init_rand_city"]

# Data
dref = utils.open_thredds(
    config["url"]["dref"]
).chunk({'time': -1}).sel(time=slice('1991', '2020'))

dsim = utils.open_thredds(config["url"]["dsim"])
dsim = xc.core.calendar.convert_calendar(dsim, 'default')

places = gpd.read_file(config["url"]["places"]).to_crs(epsg=8858)
#cities = gpd.read_file(
#    "https://pavics.ouranos.ca/geoserver/analogs/ows?"
#    "service=wfs&version=2.0.0&request=GetFeature"
#    "&typeName=analogs:cities"
#    "&propertyname=city,province,prov_code,density,the_geom&outputFormat=application/json"
#)
cities = gpd.read_file(cities_file)

# Pre-compute/download the reference distributions and the density map
if not Path('benchmark.obj').is_file():
    benchmark = utils.open_thredds(config["url"]["benchmark"]).benchmark.load()
    
    with open('benchmark.obj', 'wb') as obj_handler:
        pickle.dump(benchmark, obj_handler)
else:
    with open('benchmark.obj', 'rb') as obj_handler:
        benchmark = pickle.load(obj_handler)
    
if not Path('density.obj').is_file():
    masks = utils.open_thredds(config["url"]["masks"])

    density = masks.dens_adj.sel(year=2020).where(
        masks.roi & dref.isel(time=0).notnull().to_array().all('variable')
    ).load()

    with open('density.obj', 'wb') as obj_handler:
        pickle.dump(density, obj_handler)

# load pickled data
else:
    with open('density.obj', 'rb') as obj_handler:
        density = pickle.load(obj_handler)



In [ ]:
dash = pn.template.VanillaTemplate(title="Climate analogues of Canadian cities.")

w_city = pn.widgets.Select(
    name='Target city',
    options={f"{city.province}: {city.city}": i for i, city in cities.iterrows()}
)
if init_rand_city:
    def random_city():
        w_city.value = np.random.randint(0, len(cities))

    pn.state.onload(random_city)

w_ssp = pn.widgets.RadioButtonGroup(
    name='Emission scenario',
    options=list(dsim.ssp.values),
)
w_tgt_period = pn.widgets.DiscreteSlider(
    name='Target period',
    options={f"{x-29} - {x}": slice(f"{x-29}", f"{x}") for x in range(2020, 2101, 10)},
    value=slice("2041", "2070")
)

w_indices = pn.widgets.MultiChoice(
    name='Climate indices',
    max_items=4,
    options=[]
)

@pn.depends(icity=w_city, ssp=w_ssp, tgt_period=w_tgt_period)
def usable_indices(icity, ssp, tgt_period):
    with pn.param.set_values(w_indices, loading=True):
        unusable = search.get_unusable_indices(cities, dref, dsim, icity, ssp, tgt_period)
        options = {v.long_name: k for k, v in dsim.data_vars.items() if k not in unusable}
        values = [v for v in w_indices.value if v not in unusable]
        w_indices.options = options
        w_indices.value = values
    #if unusable:
    #    return pn.pane.Alert(
    #        "Some indices are not usable for this combination of city, scenario and target period.",
    #        alert_type='warning'
    #    )
    return pn.pane.Str("")

w_density_factor = pn.widgets.IntInput(name='Density range factor', value=4, step=1, start=2, end=10)

@pn.depends(icity=w_city, density_factor=w_density_factor)
def info(icity, density_factor):
    dens = cities.iloc[icity].density
    dmin = max(dens / density_factor, 10)
    dmax = dens * density_factor
    N = ((density < dmax) & (density > dmin)).sum().item()
    return pn.pane.Markdown(
        f"* Target population density : {dens:.0f} hab./ km²\n"
        f"* Population density range : {dmin:.0f} - {dmax:.0f} hab. / km²\n"
        f"* Number of candidate pixels : {N}"
    )

w_run = pn.widgets.Button(name="Run analogues search")

w_progress = pn.widgets.Progress(active=False, min_width=200, width=300) # Progress(active=False, delta=0.1, min_width=200, width=300)

@pn.depends(clicks=w_run.param.clicks)
def analogs_search(clicks):
    """This function does everything."""
    if clicks == 0:
        return pn.pane.Str('Please run an analogue search.')
    
    w_progress.active = True

    # Translate the widget's values to variables
    # The goal is to keep the code here and in the notebook in sync so that copy-pasting the main parts doesn't break
    icity = w_city.value
    ssp = w_ssp.value
    ssp_opts = w_ssp.options
    tgt_period = w_tgt_period.value
    periods = list(w_tgt_period.options.values())
    climate_indices = w_indices.value
    density_factor = w_density_factor.value
    max_density = w_density_factor.end
    

    ### Analogue finding begins here. Code below should be the exact same as in the notebook
    city = cities.iloc[icity]

    #sim = dsim[climate_indices].isel(location=icity).sel(ssp=ssp)
    
    analogs, sim, ref = search.analogs(dsim, 
                                              dref, 
                                              density, 
                                              benchmark, 
                                              city,cities, 
                                              climate_indices, 
                                              density_factor,max_density, 
                                              tgt_period,periods, 
                                              ssp,ssp_opts)
    #analogs, sim, ref = search.analogs_search(
    #    sim, dref[climate_indices], density, city, tgt_period, benchmark, w_density_factor.value
    #)
    selector = widgets.ColoredToggleGroup(analogs.quality)
    
    # Map of analoguea
    @pn.depends(iana=selector.param.value)
    def chosen_point(iana):
        return gv.Points([analogs.iloc[iana].geometry])
    
    analogs_lines = gpd.GeoDataFrame(
        analogs.drop(columns=['geometry']),
        geometry=[LineString([city.geometry, geom]) for geom in analogs.geometry]
    )
    
    analog_map =  pn.pane.HoloViews(
        (
            gv.tile_sources.Wikipedia
            * gv.Points([city.geometry]).opts(color=fut_col, marker='star', size=15)
            * gv.Path(analogs_lines, vdims=['simulation', 'quality', 'rank']).opts(tools=['hover'], color='gray')
            * gv.Points(analogs, vdims=['quality']).opts(color='quality', marker='circle', size=10, cmap=dict(zip(quality_terms, quality_colors)), line_color='k')
            * gv.DynamicMap(chosen_point).opts(color=ana_col, marker='circle', fill_color='none', size=20, line_width=4)
        ).opts(width=600, height=500, title='Map of analogues'),
        min_width=600
    )
    
    # Cards
    
    cards = pn.Accordion(min_width=800, width=850)
    climdict = {}
    for climind in climate_indices:
        climdict[sim[climind].long_name] = climind
        name = sim[climind].long_name
        data = pn.FlexBox(name=name,min_width=800)
        cards.append(data)
    
    @pn.depends(show=cards.param.active, iana=selector.param.value)
    def get_card_data(show,iana):
        print("Called get_card_data, ", show,iana)
        for i,panelcard in enumerate(cards.objects):
            if i not in show:
                panelcard.height = 0
        if not show:
            
            return cards
        else:
            for panelcardind in show:
                panelcard = cards.objects[panelcardind]
                panelcard.height = 900
                computation_needed = not utils.is_computed(ref)
                if computation_needed:
                    panelcard.clear()
                    panelcard.insert(0,pn.pane.Markdown("### Computing univariate statistics...",min_width=500))
                    w_progress.active = True
                                 
                utils.inplace_compute(ref)
                analog = analogs.iloc[iana]
                
                climind = climdict[panelcard.name]
                refi = ref[climind].sel(site=analog.site)
                histi = sim[climind].sel(realization=analog.simulation, time=slice('1991', '2020'))
                simi = sim[climind].sel(realization=analog.simulation, time=tgt_period)

                vmin = min(histi.mean() - 3 * histi.std(), refi.min(), simi.min())
                vmax = max(histi.mean() + 3 * histi.std(), refi.max(), simi.max(), 2 * histi.mean() - vmin)
                vmin = 2 * histi.mean() - vmax
                xlim = (float(vmin), float(vmax))

                uni_score = xa.zech_aslan(simi, refi)
                qflag = utils.get_quality_flag(uni_score, [climind], benchmark)

                units = f"[{simi.units}]"
                name = simi.long_name

                dist_diff = (
                    hv.Distribution(simi.values, label="Target's future").opts(color=fut_col)
                    * hv.Distribution(refi.values, label="Analogue's present").opts(color=ana_col)
                    * hv.Distribution(histi.values, label="Target's present").opts(color='white', line_color='black')
                ).opts(
                    yaxis=None, ylabel='dist', xlabel=name, responsive=True, aspect=3,
                    legend_cols=True, legend_offset=(0, 0), legend_position='bottom', fontscale=0.8,
                    title='Distribution comparison', xlim=xlim,
                )

                mean_change = (
                    hv.Overlay(
                        [hv.VLine((histi.mean() + n * histi.std()).item()).opts(color='pink', line_dash='dashed', alpha=0.5)
                         for n in range(-3, 4)]
                    ) * hv.Points([[refi.mean().item(), 1]], label="Analogue").opts(color=ana_col, size=20, marker='circle')
                    * hv.Points([[histi.mean().item(), 1]], label="Target's present").opts(color=hist_col, size=20, marker='star', line_color='k')
                    * hv.Points([[simi.mean().item(), 1]], label="Target's future").opts(color=fut_col, size=20, marker='star')
                ).opts(
                    yaxis=None, xlim=xlim, responsive=True, height=100, xlabel=name,
                    show_legend=False, # legend_position='bottom', legend_offset=(0, 0), legend_cols=True, 
                    fontscale=0.8, title='Average change', ylabel='nothing'
                )

                if int(tgt_period.start) >= 2020:
                    refcp = refi.assign_coords(time=simi.time).hvplot(color=ana_col, hover=False, legend=False)
                else:
                    refcp = hv.Overlay()

                timeseries = (
                    (hv.VLine(simi.indexes['time'][0]) * hv.VLine(simi.indexes['time'][-1])).opts(hv.opts.VLine(color='lightblue', line_width=2))
                    * sim[climind].hvplot(by='realization', color='lightgrey', alpha=0.5, hover=False, legend=False)
                    * sim[climind].median('realization').hvplot(color='darkgrey', hover=False, legend=False)
                    * refi.hvplot(color=ana_col, label='Selected best analogue')
                    * refcp
                    * sim[climind].sel(realization=analog.simulation).hvplot(color=fut_col, label='Selected simulation on target city')
                ).opts(
                    ylabel=units, xlabel='', title='Full timeseries', legend_position='top',
                    responsive=True, aspect=2, show_legend=False
                )
                description = pn.pane.Markdown(
                    f"### Quality of univariate analogy: {uni_score: 5.2f} ({quality_terms[qflag]})\n"
                    f'- **Description**: {simi.description}\n'
                    f'- **Units** : {simi.units}\n\n',
                    min_width = 500
                )
                holoview = pn.pane.HoloViews(hv.Layout([dist_diff, mean_change]).cols(1), linked_axes=False)
                if computation_needed:
                    w_progress.active = False
                panelcard.clear()
                panelcard.insert(0,description)
                panelcard.insert(1,holoview)
                panelcard.insert(2,timeseries)
            return cards
        
    @pn.depends(iana=selector.param.value)
    def summary(iana):
        analog = analogs.iloc[[iana]].to_crs(epsg=8858)
        distances = places.distance(analog.geometry.iloc[0])
        near_city = places.iloc[distances.argmin()].copy()
        near_dist = analog.distance(cities.loc[[city.name]].to_crs(epsg=8858).geometry.iloc[0]).iloc[0] / 1000

        if near_city.ADM0_A3 in ['USA', 'CAN']:
            near_city['fullname'] = f"{near_city['NAME']}, {near_city['ADM1NAME']}"
        else:
            near_city['fullname'] = f"{near_city['NAME']}, {near_city['ADM0_A3']}"

        
        data = {
            'Urban area': [f"{city.city}, {city.prov_code}", f"near {near_city.fullname} ({near_dist:.0f} km)"],
            'Coordinates': [f"{utils.dec2sexa(city.geometry.y)}N, {utils.dec2sexa(-city.geometry.x)}W",
                            f"{utils.dec2sexa(analogs.iloc[iana].geometry.y)}N, {utils.dec2sexa(-analogs.iloc[iana].geometry.x)}W"],
            "Time period": [f"{tgt_period.start}-{tgt_period.stop}", "1991-2020"],
            "Data source": [f"{analog.iloc[0].simulation} / SSP{ssp[3]}-{ssp[4]}.{ssp[5]}", "ERA5-Land"],
            "Pop. density": [f"{city.density:.0f} hab/km²", f"{analog.iloc[0].density:.0f} hab/km²"]
        }
        perc_fmt = '.0f' if analog.iloc[0].percentile > 1 else ('.2f' if analog.iloc[0].percentile > 0.01 else '.04f')
        return pn.Column(
            pn.pane.Markdown(
                f'### Current selection : \#{iana + 1}\n'
                f'**Quality of analogy**: {analog.iloc[0].quality} ({analog.iloc[0].score:.3f}, top {analog.iloc[0].percentile:{perc_fmt}} %)\n'
                f'**Representativeness score**: {analog.iloc[0].zscore:.2f}'
            ),
            pn.pane.DataFrame(
                pd.DataFrame.from_dict(data, orient='index', columns=['Target', 'Analogue']),
                min_width=300
            ),        
        )
    
    w_progress.active = False
    return pn.Column(
        selector,
        pn.layout.Divider(),
        pn.Row(analog_map, summary),
        get_card_data,
        max_width=1000
    )
        
dash.sidebar.append(w_city)
dash.sidebar.append(w_ssp)
dash.sidebar.append(w_tgt_period)
dash.sidebar.append(w_indices)
dash.sidebar.append(usable_indices)
dash.sidebar.append(w_density_factor)
dash.sidebar.append(info)
dash.sidebar.append(w_run)
dash.sidebar.append(w_progress)
dash.main.append(analogs_search)

To use this dashboard from within PAVICS and have it run in your user account use the first line of the next cell (`s = dash.show(...)`) and comment the second one (`dash.servable()`). In that case, if you want to update the dashboard after making changes, don't forget to run `s.stop()` before rerunning  `s = dash.show(...)`.

In [ ]:
s = dash.show(port=9092, websocket_origin='*')
#dash.servable()
print(f"The line above is lying to you. The _real_ adress is:\n https://pavics.ouranos.ca/jupyter/user-redirect/proxy/{s.port}/")

In [ ]:
s.stop()